# Imports

In [14]:
# Necessary Libraries

# API Info from ENV
from dotenv import load_dotenv
import os
import time
import zipfile
import re

import isodate
import urllib3
import pandas as pd

# Simscale Imports
from simscale_sdk import Configuration, ApiClient, ProjectsApi, StorageApi, GeometryImportsApi, GeometriesApi, \
    MeshOperationsApi, SimulationsApi, SimulationRunsApi, ReportsApi, Project, GeometryImportRequest, ApiException, \
    MaterialsApi, MaterialGroupType, MaterialUpdateRequest, MaterialUpdateOperation, MaterialUpdateOperationReference
from simscale_sdk import GeometryImportRequestLocation, GeometryImportRequestOptions, Point, DimensionalVectorLength, \
    DecimalVector
from simscale_sdk import Incompressible, IncompressibleFluidMaterials, FluidModel, FluidInitialConditions, \
    AdvancedConcepts, TopologicalReference, FluidNumerics, RelaxationFactor, DimensionalPressure, \
    ProbePointsResultControl, \
    ResidualControls, Tolerance, FluidSolvers, Schemes, TimeDifferentiationSchemes, GradientSchemes, DivergenceSchemes, \
    LaplacianSchemes, InterpolationSchemes, SurfaceNormalGradientSchemes, SlipVBC, VelocityInletBC, WallBC, SymmetryBC, NoSlipVBC, FixedValueVBC, \
    ComponentVectorFunction, ConstantFunction, PressureOutletBC, FixedValuePBC, DimensionalFunctionPressure, \
    FluidSimulationControl, DimensionalTime, TimeStepWriteControl, DimensionalArea, ScotchDecomposeAlgorithm, FluidResultControls, \
    DimensionalVectorFunctionSpeed, ForcesMomentsResultControl, ForceMomentCoefficientsResultControl, DimensionalSpeed, OneOfTimeDifferentiationSchemesForDefault
from simscale_sdk import SimulationSpec, MeshOperation, SimmetrixMeshingFluid, AutomaticLayerOn, SimulationRun
from simscale_sdk import TopViewPredefinedCameraSettings, ProjectionType, Vector3D, ModelSettings, Filters, ScalarField, \
    ResolutionInfo, ReportRequest, TimeStepAnimationOutputSettings, AnimationReportProperties, CuttingPlane, RenderMode

from onshape_client.client import Client

# API Clients and Project

In [41]:
# Check that the environment variables are set
load_dotenv()


# API client configuration
api_key = os.getenv("SIMSCALE_API_KEY")
api_url = "https://api.simscale.com"
api_key_header = "X-API-KEY"

configuration = Configuration()
configuration.debug = True
configuration.host = api_url + "/v0"
configuration.api_key = {
    api_key_header: api_key
}

# Get Main Client
api_client = ApiClient(configuration)

retry_policy = urllib3.Retry(connect=5, read=5, redirect=0, status=5, backoff_factor=0.2)
api_client.rest_client.pool_manager.connection_pool_kw["retries"] = retry_policy

# API clients
project_api = ProjectsApi(api_client)
storage_api = StorageApi(api_client)
geometry_import_api = GeometryImportsApi(api_client)
geometry_api = GeometriesApi(api_client)
mesh_operation_api = MeshOperationsApi(api_client)
simulation_api = SimulationsApi(api_client)
simulation_run_api = SimulationRunsApi(api_client)
reports_api = ReportsApi(api_client)
materials_api = MaterialsApi(api_client)

# Create project
project = Project(
    name="Incompressible AeroBody Flow", description="Incompressible flow around a test aero body.", measurement_system="SI"
)
project = project_api.create_project(project)
project_id = project.project_id
print(f"projectId: {project_id}")

# Read project information and update with the deserialized model
project = project_api.get_project(project_id)
project_api.update_project(project_id, project)

projectId: 4374360506491890050


In [42]:
access_key = "m0gYBcdS7XDGVEN6P6ZOnCKV"
secret_key = "z7yhjr1vU2CCAUTdKv8iaElUW7Y6XqMXaIlLeP06SioI7Jms"
base = "https://cuautodrone.onshape.com/"
onshape_client = Client(configuration={"base_url":base,
                               "access_key":access_key,
                               "secret_key":secret_key})

/home/whowes/CUAutoDrone/CFDAnalysis/.cfdenv/lib/python3.11/site-packages/onshape_client/client.py:157: UserWarning: A Client was already created so this will create another and override it. Please use Client.get_client() to get the previously created client.
  warnings.warn(


# Geometry

In [43]:
did = "8cc415b7a80196497cd537a4"
wid = "2d2e7365b615264c75bb21d4"
eid = "bcc58dd1c00017056413bb9d"

### Get variable info for setting 

Needs to be done once before running

In [44]:
fixed_url = "api/v9/partstudios/d/"+did+"/w/"+wid+"/e/"+eid+"/features?rollbackBarIndex=-1&includeGeometryIds=true&noSketchGeometry=false"

final_url = base+fixed_url

method = 'GET'

params = {}

payload = {}
headers = {'Accept': 'application/json;charset=UTF-8; qs=0.09',
           'Authorization': 'Basic CREDENTIALS'}

response = onshape_client.api_client.request(method, url=final_url, query_params=params, headers=headers, body=payload)

In [45]:
print(response.data)

{
  "btType" : "BTFeatureListResponse-2457",
  "serializationVersion" : "1.2.7",
  "isComplete" : true,
  "rollbackIndex" : 12,
  "features" : [ {
    "btType" : "BTMFeature-134",
    "namespace" : "",
    "name" : "###name = #value",
    "suppressed" : false,
    "parameters" : [ {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "M4lzoIgBDwyWw3tdR",
      "parameterId" : "initEntities"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "M1Bw/+t5F6lGzHJx4",
      "enumName" : "VariableMode",
      "value" : "ASSIGNED",
      "parameterId" : "mode"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "MymiKYdFyyQ+VSHLb",
      "enumName" : "VariableType",
      "value" : "LENGTH",
      "parameterId" : "variableType"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "MnzSXYWpJ8iOtAVFk",
      "enumName" : "VariableMeasurementMode",


### Set Value

This will need onshape online at first to get ids

In [46]:
feature1id = "FVJX0qmQg48MNre_0"
feature2id = "Fgz4GyXRtwO8Vli_0"
feature3id = "FTKV8r7d8hr5kme_0"
feature4id = "Fwnh8qJNpq0GQaW_0"

c1 = 30
c2 = 20
c3 = 20
c4 = 20

In [47]:
fixed_url_1 = "api/v9/partstudios/d/"+did+"/w/"+wid+"/e/"+eid+"/features/featureid/"+feature1id
fixed_url_2 = "api/v9/partstudios/d/"+did+"/w/"+wid+"/e/"+eid+"/features/featureid/"+feature2id
fixed_url_3 = "api/v9/partstudios/d/"+did+"/w/"+wid+"/e/"+eid+"/features/featureid/"+feature3id
fixed_url_4 = "api/v9/partstudios/d/"+did+"/w/"+wid+"/e/"+eid+"/features/featureid/"+feature4id


final_url_1 = base+fixed_url_1
final_url_2 = base+fixed_url_2
final_url_3 = base+fixed_url_3
final_url_4 = base+fixed_url_4

method = 'POST'

params = {}

payload1 = {"btType": "BTFeatureDefinitionCall-1406",
           "feature": {
    "featureId": feature1id,
    "featureType": "assignVariable",
    "name" : "###name = #value",
    "parameters" : [ {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "M4i57bsHsMqZhiOnI",
      "parameterId" : "initEntities"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "MnDwqZ8pWgJ2dfmAO",
      "enumName" : "VariableMode",
      "value" : "ASSIGNED",
      "parameterId" : "mode"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "MdnRrwUKjqoA00SJe",
      "enumName" : "VariableType",
      "value" : "LENGTH",
      "parameterId" : "variableType"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "M4niR/6caCjPh3Z3V",
      "enumName" : "VariableMeasurementMode",
      "value" : "DISTANCE",
      "parameterId" : "measurementMode"
    }, {
      "btType" : "BTMParameterString-149",
      "value" : "C1",
      "nodeId" : "MlSYdRkfgRt/zC3XT",
      "parameterId" : "name"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : f"{c1} mm",
      "nodeId" : "MB14S9CpxriJkcmDu",
      "parameterId" : "lengthValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0.0*deg",
      "nodeId" : "M0OXpUurDvkJ/ogvM",
      "parameterId" : "angleValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0.0",
      "nodeId" : "MnS5ahT1Lo2qcPb5J",
      "parameterId" : "numberValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0",
      "nodeId" : "MgbZwq8ScDCwRJOXk",
      "parameterId" : "anyValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : f"{c1} mm",
      "nodeId" : "M4hibMWtmLrdUu9+c",
      "parameterId" : "value"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "MQmKxDdzn24P+GO03",
      "parameterId" : "entityCouple"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "MDV/MuXmghUrw1rb4",
      "enumName" : "VariableMinMaxSelection",
      "value" : "MINIMUM",
      "parameterId" : "minmax"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "MpXuQabCYxxm/TEq6",
      "parameterId" : "extendEntities"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "M5qnoHGnkt7lZZ/1F",
      "parameterId" : "measureFromAxis"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0.0*mm",
      "nodeId" : "MTq7Qoi+lP/S1mECG",
      "parameterId" : "distance"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0.0*mm",
      "nodeId" : "MeG7vxANA9FQ+RxzA",
      "parameterId" : "xOffset"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0.0*mm",
      "nodeId" : "MffwFCh/5/VHquqHj",
      "parameterId" : "yOffset"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0.0*mm",
      "nodeId" : "M20wRAAxagRMOMl8L",
      "parameterId" : "zOffset"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "M319rpmecuFRrnvt4",
      "enumName" : "AxisWithCustom",
      "value" : "DISTANCE",
      "parameterId" : "componentSelector"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "MFEF0h0u09WBRd0cK",
      "parameterId" : "customDirection"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0.0*mm",
      "nodeId" : "MNpQGZ3qz92QBKOeo",
      "parameterId" : "customOffset"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "MoPIbARN9hr7dhiYq",
      "parameterId" : "lengthEntities"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "MEPg0c7fvtu7vt2EE",
      "parameterId" : "radius"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "MUpN3L0nZipM7aL2h",
      "parameterId" : "diameterEntity"
    }, {
      "btType" : "BTMParameterString-149",
      "value" : "",
      "nodeId" : "MYgUNe9tu/vo5P81s",
      "parameterId" : "description"
    }]}}

payload2 = {"btType": "BTFeatureDefinitionCall-1406",
           "feature": 
     {
    "featureId": feature2id,
    "featureType": "assignVariable",
    "name" : "###name = #value",
    "parameters" : [ {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "LeVAKpOBwYMm0O+D",
      "parameterId" : "initEntities"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "efqcuSITuvGa3uCk",
      "enumName" : "VariableMode",
      "value" : "ASSIGNED",
      "parameterId" : "mode"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "VFElz2Mb97miElG1",
      "enumName" : "VariableType",
      "value" : "LENGTH",
      "parameterId" : "variableType"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "8t/mJttKHwwtnalj",
      "enumName" : "VariableMeasurementMode",
      "value" : "DISTANCE",
      "parameterId" : "measurementMode"
    }, {
      "btType" : "BTMParameterString-149",
      "value" : "C2",
      "nodeId" : "MPRMuQc4NYqIFfVvY",
      "parameterId" : "name"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : f"{c2} mm",
      "nodeId" : "YONEPJvvIozbMAwG",
      "parameterId" : "lengthValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 deg",
      "nodeId" : "uwZEmgN+B3y+22JP",
      "parameterId" : "angleValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0",
      "nodeId" : "BLPoZv2BLa9IrY6u",
      "parameterId" : "numberValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0",
      "nodeId" : "Hf1icb5MR+/cFT8l",
      "parameterId" : "anyValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : f"{c2} mm",
      "nodeId" : "vq1f98WpR6mvwTSZ",
      "parameterId" : "value"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "oUW0ksgILOhMvWf/",
      "parameterId" : "entityCouple"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "AM6ZsodiqFEsmr5S",
      "enumName" : "VariableMinMaxSelection",
      "value" : "MINIMUM",
      "parameterId" : "minmax"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "jAbaproI/mLrCAag",
      "parameterId" : "extendEntities"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "nw0gDK1XVJHuwoz2",
      "parameterId" : "measureFromAxis"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "eLkMvZOUuVOmsSBf",
      "parameterId" : "distance"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "MMyKz6zBWFpMh4is",
      "parameterId" : "xOffset"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "EM5Iv4Dk5P7DgHnf",
      "parameterId" : "yOffset"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "pEknhHaQZX0+St4s",
      "parameterId" : "zOffset"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "AoLUtnbcGpzIcEmk",
      "enumName" : "AxisWithCustom",
      "value" : "DISTANCE",
      "parameterId" : "componentSelector"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "T84yyG3nLYk9WxOQ",
      "parameterId" : "customDirection"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "HC3Y45shILWDNuFb",
      "parameterId" : "customOffset"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "k92469LEHs+/NSYV",
      "parameterId" : "lengthEntities"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "t/kAAm693PjxmE3h",
      "parameterId" : "radius"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "BiU9576ZZJSlQnZE",
      "parameterId" : "diameterEntity"
    }, {
      "btType" : "BTMParameterString-149",
      "value" : "",
      "nodeId" : "PEs3FvY4FWyZ7Avp",
      "parameterId" : "description"
    } ]}}

payload3 = {"btType": "BTFeatureDefinitionCall-1406",
           "feature": {
          "featureId": feature3id,
          "featureType": "assignVariable",
          "name" : "###name = #value",
    "parameters" : [ {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "wmIrfahT+XG3s05b",
      "parameterId" : "initEntities"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "gPJfrh3Jnh8bqWmL",
      "enumName" : "VariableMode",
      "value" : "ASSIGNED",
      "parameterId" : "mode"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "mW+gV5y3qXmEqIW5",
      "enumName" : "VariableType",
      "value" : "LENGTH",
      "parameterId" : "variableType"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "y3MvJMV1gaO+suNg",
      "enumName" : "VariableMeasurementMode",
      "value" : "DISTANCE",
      "parameterId" : "measurementMode"
    }, {
      "btType" : "BTMParameterString-149",
      "value" : "C3",
      "nodeId" : "MU3MuVRzP9YWeyGlS",
      "parameterId" : "name"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : f"{c3} mm",
      "nodeId" : "PDQ73MoVR8jEDrGm",
      "parameterId" : "lengthValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 deg",
      "nodeId" : "GjOihFJXXUKuGz0W",
      "parameterId" : "angleValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0",
      "nodeId" : "qolu/2W9Hy0N0ZaW",
      "parameterId" : "numberValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0",
      "nodeId" : "9sHqBs+OznQ4c+ao",
      "parameterId" : "anyValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : f"{c3} mm",
      "nodeId" : "PhLgclOLU+F+fQE0",
      "parameterId" : "value"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "ANVjbHXhcbia7aW4",
      "parameterId" : "entityCouple"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "sQAeeQ0n0SI2IyP+",
      "enumName" : "VariableMinMaxSelection",
      "value" : "MINIMUM",
      "parameterId" : "minmax"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "bP4K80Kj/nE6993T",
      "parameterId" : "extendEntities"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "+GJM4XFV7iic9AJ6",
      "parameterId" : "measureFromAxis"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "KuiUEWiFspKgKXPQ",
      "parameterId" : "distance"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "uQaPfCJGEHcSiuvk",
      "parameterId" : "xOffset"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "wXK68rF2aUi7oMmp",
      "parameterId" : "yOffset"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "Ju9oOe8otb3OMFle",
      "parameterId" : "zOffset"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "3fOKpNi+RAN1szoH",
      "enumName" : "AxisWithCustom",
      "value" : "DISTANCE",
      "parameterId" : "componentSelector"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "+ZlVN54sAFIEtn9E",
      "parameterId" : "customDirection"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "Y4syxfMcmWel5A+x",
      "parameterId" : "customOffset"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "Z1LYXiT3S9f5gKvz",
      "parameterId" : "lengthEntities"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "a5v+7Wzx78Coja1E",
      "parameterId" : "radius"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "30Ir150ViFpGUwsb",
      "parameterId" : "diameterEntity"
    }, {
      "btType" : "BTMParameterString-149",
      "value" : "",
      "nodeId" : "9pRdiCWYcT1CE7Ni",
      "parameterId" : "description"
    } ]}}

payload4 = {"btType": "BTFeatureDefinitionCall-1406",
           "feature":{
          "featureId": feature4id,
          "featureType": "assignVariable",
          "name" : "###name = #value",
    "parameters" : [ {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "1VXDZppKajw6gZ60",
      "parameterId" : "initEntities"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "AE27ZTopyRbw6GTj",
      "enumName" : "VariableMode",
      "value" : "ASSIGNED",
      "parameterId" : "mode"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "xPizGQ1ZTn/CRbZK",
      "enumName" : "VariableType",
      "value" : "LENGTH",
      "parameterId" : "variableType"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "ksvonksFwT7WvPtY",
      "enumName" : "VariableMeasurementMode",
      "value" : "DISTANCE",
      "parameterId" : "measurementMode"
    }, {
      "btType" : "BTMParameterString-149",
      "value" : "C4",
      "nodeId" : "MZ/T0CQNnzOZPIoKD",
      "parameterId" : "name"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : f"{c4} mm",
      "nodeId" : "nUHGCRq2HNYv43lZ",
      "parameterId" : "lengthValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 deg",
      "nodeId" : "EADGAQIWvQEcf24l",
      "parameterId" : "angleValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0",
      "nodeId" : "lhld0LnypJ+KCHWk",
      "parameterId" : "numberValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0",
      "nodeId" : "jTNHk3exs2cQAA6h",
      "parameterId" : "anyValue"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : f"{c4} mm",
      "nodeId" : "c/ucxw0j3vNfZb1s",
      "parameterId" : "value"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "ifNtKb66A65V/igt",
      "parameterId" : "entityCouple"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "VNc/1yfgQC/EEhM2",
      "enumName" : "VariableMinMaxSelection",
      "value" : "MINIMUM",
      "parameterId" : "minmax"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "7W5Fu3MMQyXGfP0b",
      "parameterId" : "extendEntities"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "DcvnZMYmhXjRqlSd",
      "parameterId" : "measureFromAxis"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "rLIf6dfDATMC3vYK",
      "parameterId" : "distance"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "15bjyJ+vRinHXNgT",
      "parameterId" : "xOffset"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "ZCvMGwIcGO2Kz7ga",
      "parameterId" : "yOffset"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "VJxhys5qViGm0aqU",
      "parameterId" : "zOffset"
    }, {
      "btType" : "BTMParameterEnum-145",
      "namespace" : "",
      "nodeId" : "dn0CtGoDF0NYXZEr",
      "enumName" : "AxisWithCustom",
      "value" : "DISTANCE",
      "parameterId" : "componentSelector"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "zUM2oAHcKG6vdRqP",
      "parameterId" : "customDirection"
    }, {
      "btType" : "BTMParameterQuantity-147",
      "isInteger" : False,
      "value" : 0.0,
      "units" : "",
      "expression" : "0 mm",
      "nodeId" : "dcUOTAAZXcJChgyV",
      "parameterId" : "customOffset"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "e6MnX/Eig4OzVgcJ",
      "parameterId" : "lengthEntities"
    }, {
      "btType" : "BTMParameterBoolean-144",
      "value" : False,
      "nodeId" : "TJG/vi8jTlM/tBG+",
      "parameterId" : "radius"
    }, {
      "btType" : "BTMParameterQueryList-148",
      "queries" : [ ],
      "nodeId" : "BkaXzDrEEf/z0jQ0",
      "parameterId" : "diameterEntity"
    }, {
      "btType" : "BTMParameterString-149",
      "value" : "",
      "nodeId" : "oZCPvkdJgMpN6CZC",
      "parameterId" : "description"
    } ]}}

headers = {'Accept': 'application/json;charset=UTF-8; qs=0.09',
           'Authorization': 'Basic CREDENTIALS',
           'Content-Type': "application/json;charset=UTF-8; qs=0.09"}

_ = onshape_client.api_client.request(method, url=final_url_1, query_params=params, headers=headers, body=payload1)
_ = onshape_client.api_client.request(method, url=final_url_2, query_params=params, headers=headers, body=payload2)
_ = onshape_client.api_client.request(method, url=final_url_3, query_params=params, headers=headers, body=payload3)
_ = onshape_client.api_client.request(method, url=final_url_4, query_params=params, headers=headers, body=payload4)

In [48]:
method = "GET"
final_url = base+"api/v9/variables/d/"+did+"/w/"+wid+"/e/"+eid+"/variables" 
params = {"includeValuesAndReferencedVariables":True}
headers = {'Accept': 'application/json;charset=UTF-8; qs=0.09',
           'Authorization': 'Basic CREDENTIALS',
           'Content-Type': "application/json;charset=UTF-8; qs=0.09"}

response = onshape_client.api_client.request(method, url=final_url, query_params=params,headers=headers,body={})

In [49]:
print(response.data)

[ {
  "variableStudioReference" : {
    "referenceVersionId" : "",
    "referenceElementId" : "efa77854ab717c2531bdc8ba",
    "referenceDocumentId" : "",
    "variableNames" : [ ],
    "entireVariableStudio" : true
  },
  "variables" : [ ]
}, {
  "variableStudioReference" : null,
  "variables" : [ {
    "type" : "LENGTH",
    "name" : "C1",
    "value" : "0.03 meter",
    "expression" : "30 mm",
    "description" : ""
  }, {
    "type" : "LENGTH",
    "name" : "C2",
    "value" : "0.02 meter",
    "expression" : "20 mm",
    "description" : ""
  }, {
    "type" : "LENGTH",
    "name" : "C3",
    "value" : "0.02 meter",
    "expression" : "20 mm",
    "description" : ""
  }, {
    "type" : "LENGTH",
    "name" : "C4",
    "value" : "0.02 meter",
    "expression" : "20 mm",
    "description" : ""
  }, {
    "type" : "LENGTH",
    "name" : "Xdiameter",
    "value" : "0.04166541952621774 meter",
    "expression" : null,
    "description" : ""
  } ]
} ]


In [50]:
import numpy as np
index = response.data.index('"name" : "Xdiameter"')
diameter_string = response.data[index:]
my_match = re.search(r'"value" : ".*"', diameter_string)
measured_radius = float(my_match.group(0)[12:-7])/2
print(measured_radius)

reference_area = np.pi * (measured_radius**2)

0.02083270976310887


In [51]:
print(reference_area)

0.001363456854190639


### Download Onshape File

In [52]:
## First Get parts to get name of part you want to download

fixed_url = "api/v9/parts/d/"+did+"/w/"+wid+"/e/"+eid


final_url = base+fixed_url

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/json;charset=UTF-8; qs=0.09', 'Authorization': 'Basic CREDENTIALS'}

response = onshape_client.api_client.request(method, url=final_url, query_params=params, headers=headers, body=payload)
response_string = response.data

In [53]:
print(response_string)


[ {
  "name" : "Part 1",
  "state" : "IN_PROGRESS",
  "appearance" : {
    "color" : {
      "green" : 165,
      "blue" : 165,
      "red" : 165
    },
    "isGenerated" : true,
    "opacity" : 255
  },
  "description" : null,
  "revision" : null,
  "microversionId" : "fd655cdfc32da2aca681ebf8",
  "partNumber" : null,
  "elementId" : "bcc58dd1c00017056413bb9d",
  "partId" : "JKD",
  "bodyType" : "solid",
  "partQuery" : "query=qCompressed(1.0,\"&1bb$eJx9UMluwjAQ/ZnpsQgHh8IxiwNRSwJ2qpJTZCcuWCx2HUctf9+ESL1U4jjzZt72FPqw66S9bXyGPGhUyy9CHTrulL7G3PEAbbz/SHEzkqEZ5DRdpVnwVsVkS7KYZFHJlqCtOqgrP7cBjqY9MQd5dcrdhq+QA/kbGAYSrwgTcFSt03ZcLiCiJCjSPOsBbaS9S6ZN6EHaBIghNJlDkp+kWbRhl+DABkmFvpWVuenVv4Y4d6IG2CsponVFsiItSiahPUlXH0cDacPqnmjvDqzuNpepxEJo5/Rl8Ewx9emcvtAF5VTQmjbsGXmTGThtHuAYzvLTPTjwoe/m6CiO5n34MI/L8SgSY7AlJLs1muK9UvR9RmpTIW3Ij7NdI6nom2QfZFtUw+Mv6iKOjQ==\",id);",
  "partIdentity" : null,
  "meshState" : "NO_MESH",
  "isHidden" : false,
  "configurationId" : "",
  "isFlattenedBody" : false,
  "thu

In [54]:
index = response_string.index('"name" : "Part 1"')
aero_shell_string = response_string[index:]
my_match = re.search(r'"partId" : ".*"', aero_shell_string)
pid = my_match.group(0)[12:-1]
print(pid)

JKD


In [55]:
fixed_url = "api/v9/parts/d/"+did+"/w/"+wid+"/e/"+eid+"/partid/"+pid+"/parasolid"


cad_final_url = base+fixed_url

method = 'GET'

params = {}
payload = {}
headers = {'accept': '*/*'}

response = onshape_client.api_client.request(method, url=cad_final_url, query_params=params, headers=headers, body=payload)

/home/whowes/CUAutoDrone/CFDAnalysis/.cfdenv/lib/python3.11/site-packages/onshape_client/apikey_headers.py:33: UserWarning: Requesting a url from a different domain (cad-usw2.onshape.com:443) than in the client configuration: (cuautodrone.onshape.com)
  warnings.warn(


In [56]:
filename = "aero_body.x_t"
with open(filename, mode="w") as file:
  file.write(response.data)

In [57]:
# Upload CAD
storage = storage_api.create_storage()
with open(filename, "rb") as file:
    api_client.rest_client.PUT(url=storage.url, headers={"Content-Type": "application/octet-stream"}, body=file.read())
storage_id = storage.storage_id
print(f"storage_id: {storage_id}")

# Import CAD
geometry_import = GeometryImportRequest(
    name="Aero-Body-Geometry",
    location=GeometryImportRequestLocation(storage_id),
    format="PARASOLID",
    input_unit="m",
    options=GeometryImportRequestOptions(facet_split=False, sewing=False, improve=True, optimize_for_lbm_solver=False),
)

geometry_import = geometry_import_api.import_geometry(project_id, geometry_import)
geometry_import_id = geometry_import.geometry_import_id
geometry_import_start = time.time()
while geometry_import.status not in ("FINISHED", "CANCELED", "FAILED"):
    # adjust timeout for larger geometries
    if time.time() > geometry_import_start + 900:
        raise TimeoutError()
    time.sleep(10)
    geometry_import = geometry_import_api.get_geometry_import(project_id, geometry_import_id)
    print(f"Geometry import status: {geometry_import.status}")
geometry_id = geometry_import.geometry_id
print(f"geometry_id: {geometry_id}")

# Read geometry information and update with the deserialized model
geometry = geometry_api.get_geometry(project_id, geometry_id)
geometry_api.update_geometry(project_id, geometry_id, geometry)


storage_id: upload/d7b92d69-6870-4d44-b3f8-5d81b89d9e58
Geometry import status: FINISHED
geometry_id: 2a29accd-3fb8-4a07-982c-c9df4c311bef


In [58]:
# Look at the name of the faces
geometry_api.get_geometry_mappings(project_id, geometry_id,)

{'embedded': [{'_class': 'face',
               'name': 'B1_TE1',
               'originate_from': [{'attribute_list': [{'attribute': 'SDL/TYSA_COLOUR',
                                                       'value': '[0.6470588235294119 '
                                                                '0.6470588235294119 '
                                                                '0.6470588235294119]'}],
                                   'body': '9-0-3',
                                   'entity': '9-2-2',
                                   'path': []}]},
              {'_class': 'vertex',
               'name': 'B1_TE100',
               'originate_from': [{'attribute_list': [],
                                   'body': '9-0-3',
                                   'entity': '9-2-0',
                                   'path': []}]},
              {'_class': 'vertex',
               'name': 'B1_TE104',
               'originate_from': [{'attribute_list': [],
                   

### Get Mappings

This will need Simscale Online for the start to get names

In [59]:
# Get faces for boundary conditions

# Get geometry mappings
def get_single_entity_name(project_id, geometry_id, **kwargs):
    entities = geometry_api.get_geometry_mappings(project_id, geometry_id, **kwargs).embedded
    if len(entities) == 1:
        return entities[0].name
    else:
        raise Exception(f"Found {len(entities)} entities instead of 1: {entities}")
    
inlet_face = get_single_entity_name(project_id, geometry_id, entities=["9-2-2"])
print(inlet_face)
outlet_face = get_single_entity_name(project_id, geometry_id, entities=["9-1-2"])
aero_face = get_single_entity_name(project_id, geometry_id, entities=["10-0-2"])
volume = get_single_entity_name(project_id, geometry_id, entities = ["region0-9-0-3"])
symmetry_walls = list(map(lambda name : get_single_entity_name(project_id, geometry_id, entities=[name]), ["10-1-2", "10-2-2"]))
other_wall_faces = list(map(lambda name : get_single_entity_name(project_id, geometry_id, entities=[name]), ["9-3-2", "9-5-2"]))

B1_TE1


## Define Model

#### Define Boundary Conditions

Will need simscale online to get orientation correct

In [ ]:
drone_airspeed = 0.5 #m/s

inlet_bc = VelocityInletBC(
            name="Velocity Inlet",
            velocity=FixedValueVBC(
                value=DimensionalVectorFunctionSpeed(
                    value=ComponentVectorFunction(
                        x=ConstantFunction(value=0), y=ConstantFunction(value=-1*drone_airspeed), z=ConstantFunction(value=0)
                    )
                )
            ),
            topological_reference=TopologicalReference(entities=[inlet_face]),
        )

outlet_bc = PressureOutletBC(
            name="Pressure Outlet",
            gauge_pressure=FixedValuePBC(value=DimensionalFunctionPressure(value=ConstantFunction(value=0), unit="Pa")),
            topological_reference=TopologicalReference(entities=[outlet_face]),
        )

symmetry_bc = SymmetryBC(
            name="Symmetry Walls",
            topological_reference=TopologicalReference(entities=symmetry_walls)
)

no_slip_aerofoil = WallBC(name = "No Slip Aerofoil", velocity = NoSlipVBC(turbulence_wall="WALL_FUNCTION"), topological_reference = TopologicalReference(entities = [aero_face]))

slip_wall_bcs = WallBC(name = "Slipping walls", velocity = SlipVBC(), topological_reference = TopologicalReference(entities = other_wall_faces))

bound_conditions = [inlet_bc, outlet_bc, symmetry_bc, no_slip_aerofoil, slip_wall_bcs]

In [35]:
model = Incompressible(
    model=FluidModel(),
    initial_conditions=FluidInitialConditions(),
    advanced_concepts=AdvancedConcepts(),
    materials=IncompressibleFluidMaterials(),
    numerics=FluidNumerics(
        relaxation_factor=RelaxationFactor(),
        pressure_reference_value=DimensionalPressure(value=0, unit="Pa"),
        residual_controls=ResidualControls(
            velocity=Tolerance(),
            pressure=Tolerance(),
            turbulent_kinetic_energy=Tolerance(),
            omega_dissipation_rate=Tolerance(),
        ),
        solvers=FluidSolvers(),
        schemes=Schemes(
            time_differentiation=TimeDifferentiationSchemes(
              for_default = OneOfTimeDifferentiationSchemesForDefault(type = 'STEADYSTATE')),
            gradient=GradientSchemes(),
            divergence=DivergenceSchemes(),
            laplacian=LaplacianSchemes(),
            interpolation=InterpolationSchemes(),
            surface_normal_gradient=SurfaceNormalGradientSchemes(),
        ),
    ),
    boundary_conditions=bound_conditions,
    simulation_control=FluidSimulationControl(
        end_time=DimensionalTime(value=100, unit="s"),
        delta_t=DimensionalTime(value=1, unit="s"),
        write_control=TimeStepWriteControl(write_interval=20),
        max_run_time=DimensionalTime(value=10000, unit="s"),
        decompose_algorithm=ScotchDecomposeAlgorithm(),
    ),
    result_control = FluidResultControls(forces_moments=[
            ForceMomentCoefficientsResultControl(
                name="Forces and moments coefficients",
                lift_direction = DimensionalVectorLength(value = DecimalVector(x=0.0,y=0.0,z=1.0), unit = "m"), #based on input velociy vector and orientation of design
                drag_direction = DimensionalVectorLength(value = DecimalVector(x=0.0,y=-1.0,z=0.0), unit = "m"), #based on input velociy vector and orientation of design
                freestream_velocity_magnitude = DimensionalSpeed(value = drone_airspeed, unit = "m/s"), #drone airspeed
                write_control=TimeStepWriteControl(write_interval=1),
                topological_reference=TopologicalReference(entities=[aero_face]),
                reference_area_value = DimensionalArea(value = reference_area, unit = "m^2")
            )
        ]),
)

In [36]:
simulation_spec = SimulationSpec(name="Incompressible AeroBody", geometry_id=geometry_id, model=model)
# Create simulation
simulation_id = simulation_api.create_simulation(project_id, simulation_spec).simulation_id
print(f"simulation_id: {simulation_id}")

simulation_id: efded6bb-216f-45e0-9a09-b222002f9286


### Define Air

In [193]:
# Add a material to the simulation
material_groups = materials_api.get_material_groups().embedded
default_material_group = next((group for group in material_groups if group.group_type == MaterialGroupType.SIMSCALE_DEFAULT), None)
if not default_material_group:
    raise Exception(f"Couldn't find default material group in {material_groups}")

default_materials = materials_api.get_materials(material_group_id=default_material_group.material_group_id).embedded
material_air = next((material for material in default_materials if material.name == "Air"), None)
if not material_air:
    raise Exception(f"Couldn't find default Air material in {default_materials}")

material_data = materials_api.get_material_data(
    material_group_id=default_material_group.material_group_id,
    material_id=material_air.id
)
material_update_request = MaterialUpdateRequest(
    operations=[
        MaterialUpdateOperation(
            path="/materials/fluids",
            material_data=material_data,
            reference=MaterialUpdateOperationReference(
                material_group_id=default_material_group.material_group_id,
                material_id=material_air.id
            )
        )
    ]
)
material_update_response = simulation_api.update_simulation_materials(project_id, simulation_id, material_update_request)

# Add assignments to the new material
simulation_spec = simulation_api.get_simulation(project_id, simulation_id)
simulation_spec.model.materials.fluids[0].topological_reference = TopologicalReference(entities=[volume])
simulation_api.update_simulation(project_id, simulation_id, simulation_spec)

### Mesh

In [194]:
# Start of mesh operation
mesh_operation = mesh_operation_api.create_mesh_operation(
    project_id,
    MeshOperation(
        name="AeroBody Mesh",
        geometry_id=geometry_id,
        model=SimmetrixMeshingFluid(physics_based_meshing=True, automatic_layer_settings=AutomaticLayerOn()),
    ),
)
mesh_operation_api.update_mesh_operation(project_id, mesh_operation.mesh_operation_id, mesh_operation)
mesh_operation_id = mesh_operation.mesh_operation_id

mesh_check = mesh_operation_api.check_mesh_operation_setup(project_id, mesh_operation_id, simulation_id=simulation_id)
warnings = [entry for entry in mesh_check.entries if entry.severity == "WARNING"]
print(f"Meshing check warnings: {warnings}")
errors = [entry for entry in mesh_check.entries if entry.severity == "ERROR"]
if errors:
    raise Exception("Meshing check failed", mesh_check)

# Estimate Mesh operation
try:
    mesh_estimation = mesh_operation_api.estimate_mesh_operation(project_id, mesh_operation_id)
    print(f"Mesh operation estimation: {mesh_estimation}")

    if mesh_estimation.compute_resource is not None and mesh_estimation.compute_resource.value > 10.0:
        raise Exception("Too expensive", mesh_estimation)

    if mesh_estimation.duration is not None:
        mesh_max_runtime = isodate.parse_duration(mesh_estimation.duration.interval_max).total_seconds()
        mesh_max_runtime = max(3600, mesh_max_runtime * 2)
    else:
        mesh_max_runtime = 36000
        print(f"Mesh operation estimated duration not available, assuming max runtime of {mesh_max_runtime} seconds")
except ApiException as ae:
    if ae.status == 422:
        mesh_max_runtime = 36000
        print(f"Mesh operation estimation not available, assuming max runtime of {mesh_max_runtime} seconds")
    else:
        raise ae

mesh_operation_api.start_mesh_operation(project_id, mesh_operation_id, simulation_id=simulation_id)

# Wait until the meshing operation is complete
mesh_operation = mesh_operation_api.get_mesh_operation(project_id, mesh_operation_id)
mesh_operation_start = time.time()
while mesh_operation.status not in ("FINISHED", "CANCELED", "FAILED"):
    if time.time() > mesh_operation_start + mesh_max_runtime:
        raise TimeoutError()
    time.sleep(30)
    mesh_operation = mesh_operation_api.get_mesh_operation(project_id, mesh_operation_id)
    print(f"Meshing run status: {mesh_operation.status} - {mesh_operation.progress}")

mesh_operation = mesh_operation_api.get_mesh_operation(project_id, mesh_operation_id)
print(f"final mesh_operation: {mesh_operation}")

# Get the simulation spec and update it with mesh_id from the previous mesh operation
simulation_spec = simulation_api.get_simulation(project_id, simulation_id)
simulation_spec.mesh_id = mesh_operation.mesh_id
simulation_api.update_simulation(project_id, simulation_id, simulation_spec)


Meshing check warnings: []
Mesh operation estimation: {'cell_count': {'interval_max': 784492,
                'interval_min': 522995,
                'type': 'CELLS',
                'value': 653744},
 'compute_resource': {'interval_max': 0.37,
                      'interval_min': 0.0,
                      'type': 'CPU_HOURS',
                      'value': 0.17},
 'duration': {'interval_max': 'PT11M', 'interval_min': 'PT0S', 'value': 'PT5M'},
 'total_run_count': 1}
Meshing run status: QUEUED - 0.0
Meshing run status: QUEUED - 0.0
Meshing run status: RUNNING - 0.0
Meshing run status: RUNNING - 0.0
Meshing run status: RUNNING - 0.0
Meshing run status: RUNNING - 0.7
Meshing run status: RUNNING - 0.7
Meshing run status: RUNNING - 0.7
Meshing run status: RUNNING - 0.7
Meshing run status: RUNNING - 0.7
Meshing run status: RUNNING - 0.7
Meshing run status: RUNNING - 0.95
Meshing run status: RUNNING - 0.95
Meshing run status: RUNNING - 1.0
Meshing run status: FINISHED - 1.0
final mesh_opera

# Run Simulation

In [195]:
# Check simulation
check = simulation_api.check_simulation_setup(project_id, simulation_id)
warnings = [entry for entry in check.entries if entry.severity == "WARNING"]
print(f"Simulation check warnings: {warnings}")
errors = [entry for entry in check.entries if entry.severity == "ERROR"]
if errors:
    raise Exception("Simulation check failed", check)

# Estimate simulation
try:
    estimation = simulation_api.estimate_simulation_setup(project_id, simulation_id)
    print(f"Simulation estimation: {estimation}")

    if estimation.compute_resource is not None and estimation.compute_resource.value > 10.0:
        raise Exception("Too expensive", estimation)

    if estimation.duration is not None:
        max_runtime = isodate.parse_duration(estimation.duration.interval_max).total_seconds()
        max_runtime = max(3600, max_runtime * 2)
    else:
        max_runtime = 36000
        print(f"Simulation estimated duration not available, assuming max runtime of {max_runtime} seconds")
except ApiException as ae:
    if ae.status == 422:
        max_runtime = 36000
        print(f"Simulation estimation not available, assuming max runtime of {max_runtime} seconds")
    else:
        raise ae

# Create simulation run
simulation_run = SimulationRun(name="Run 1")
simulation_run = simulation_run_api.create_simulation_run(project_id, simulation_id, simulation_run)
run_id = simulation_run.run_id
print(f"runId: {run_id}")

# Read simulation run and update with the deserialized model
simulation_run = simulation_run_api.get_simulation_run(project_id, simulation_id, run_id)
simulation_run_api.update_simulation_run(project_id, simulation_id, run_id, simulation_run)

# Start simulation run and wait until it's finished
simulation_run_api.start_simulation_run(project_id, simulation_id, run_id)
simulation_run = simulation_run_api.get_simulation_run(project_id, simulation_id, run_id)
simulation_run_start = time.time()
while simulation_run.status not in ("FINISHED", "CANCELED", "FAILED"):
    if time.time() > simulation_run_start + max_runtime:
        raise TimeoutError()
    time.sleep(30)
    simulation_run = simulation_run_api.get_simulation_run(project_id, simulation_id, run_id)
    print(f"Simulation run status: {simulation_run.status} - {simulation_run.progress}")

Simulation check warnings: []
Simulation estimation: {'cell_count': None,
 'compute_resource': {'interval_max': 0.76,
                      'interval_min': 0.36,
                      'type': 'CPU_HOURS',
                      'value': 0.56},
 'duration': {'interval_max': 'PT11M', 'interval_min': 'PT5M', 'value': 'PT8M'},
 'total_run_count': 1}
runId: 43784a29-bee8-4156-bf37-7613c6a89634
Simulation run status: QUEUED - 0.0
Simulation run status: QUEUED - 0.0
Simulation run status: RUNNING - 0.0
Simulation run status: RUNNING - 0.0
Simulation run status: RUNNING - 0.0
Simulation run status: RUNNING - 0.0
Simulation run status: RUNNING - 0.0
Simulation run status: RUNNING - 0.0
Simulation run status: RUNNING - 0.27000001072883606
Simulation run status: RUNNING - 0.5
Simulation run status: RUNNING - 0.7799999713897705
Simulation run status: RUNNING - 1.0
Simulation run status: RUNNING - 1.0
Simulation run status: RUNNING - 1.0
Simulation run status: FINISHED - 1.0


# Analyze Results

In [196]:
# Download solution fields
solution_fields_results = simulation_run_api.get_simulation_run_results(
    project_id,
    simulation_id,
    run_id,
    page=1,
    limit=100,
    category="SOLUTION"
)
print(f"Solution results: {solution_fields_results}")

Solution results: {'embedded': [{'category': 'SOLUTION',
               'direction': None,
               'download': {'compression': 'ZIP64',
                            'format': 'OPEN_FOAM',
                            'uncompressed_size_in_bytes': 196904960,
                            'url': 'https://www.simscale.com/api/v2/load/projects/5037159088272534344/results/dde69086-f233-47e9-a2a5-187ade99a4d2/components/42459e80-56ce-4660-8c52-c629aa8caa9d/solutions?token=4f75a30f720b526271cee6a26daf75261e82fb54f3f38b10adc5ce822f3cca3820241022T051000.000Z&uid=347994158'},
               'modified_at': datetime.datetime(2024, 10, 22, 3, 12, 18, 736241, tzinfo=tzutc()),
               'result_id': '42459e80-56ce-4660-8c52-c629aa8caa9d',
               'type': 'SOLUTION_FIELD'}],
 'links': {'_self': {'href': '/projects/5037159088272534344/simulations/3a6c005c-832f-42d7-bc06-6fe4e848db29/runs/43784a29-bee8-4156-bf37-7613c6a89634/results?page=1&limit=100'},
           'first': {'href': '/proje

In [197]:
# Download force coefficient solution fields
force_coef_solution_fields_results = simulation_run_api.get_simulation_run_results(
    project_id,
    simulation_id,
    run_id,
    page=1,
    limit=100,
    category="FORCE_COEFFICIENTS_PLOT"
)
print(f"Solution Force Coefficient results: {force_coef_solution_fields_results}")

force_info = force_coef_solution_fields_results.embedded[0]
force_data_response = api_client.rest_client.GET(
    url=force_info.download.url,
    headers={api_key_header: api_key},
    _preload_content=False,
)

force_data_csv = force_data_response.data.decode("utf-8")
print(f"Area average data as CSV: {force_data_csv}")

with open("force_coef_data.csv", "w") as f:
  f.write(force_data_csv)

df = pd.read_csv("force_coef_data.csv")
drag_coef_column = df["FORCE_COEFFICIENT_CD"]
drag_coef = drag_coef_column.tail(n = 1).item()
lift_coef_column = df["FORCE_COEFFICIENT_CL"]
lift_coef = lift_coef_column.tail(n = 1).item()

print(drag_coef)
print(lift_coef)

Solution Force Coefficient results: {'embedded': [{'category': 'FORCE_COEFFICIENTS_PLOT',
               'direction': None,
               'download': {'compression': 'NONE',
                            'format': 'CSV',
                            'uncompressed_size_in_bytes': 30720,
                            'url': 'https://www.simscale.com/api/v2/load/projects/5037159088272534344/results/dde69086-f233-47e9-a2a5-187ade99a4d2/components/7b9265cd-fe34-4520-b58e-2c9386a6f1bc/plots?token=4f75a30f720b526271cee6a26daf75261e82fb54f3f38b10adc5ce822f3cca3820241022T051000.000Z&uid=347994158'},
               'modified_at': datetime.datetime(2024, 10, 22, 3, 12, 18, 735573, tzinfo=tzutc()),
               'name': 'Forces and moments coefficients',
               'quantity': None,
               'result_id': '7b9265cd-fe34-4520-b58e-2c9386a6f1bc',
               'type': 'PLOT'}],
 'links': {'_self': {'href': '/projects/5037159088272534344/simulations/3a6c005c-832f-42d7-bc06-6fe4e848db29/runs/43